<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/YOLO_v5_Wheet_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
import cv2
import tqdm
import torch
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Clone YOLOv5

Clone the YOLOv5 repository, and move all the yolo files to your current working directory

In [2]:
!git clone https://github.com/ultralytics/yolov5
!mv yolov5/* ./

Cloning into 'yolov5'...
remote: Enumerating objects: 15978, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 15978 (delta 89), reused 118 (delta 75), pack-reused 15831
Receiving objects: 100% (15978/15978), 14.61 MiB | 21.49 MiB/s, done.
Resolving deltas: 100% (10958/10958), done.


In [7]:
!pip install -r requirements.txt
clear_output()

In [16]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle competitions download global-wheat-detection
!unzip \*.zip && rm *.zip
clear_output()

In [19]:
df = pd.read_csv('/content/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]
df

,image_id,x,y,w,h,x_center,y_center,classes
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0
2,b6ab77fd7,377.0,504.0,74.0,160.0,414.0,584.0,0
3,b6ab77fd7,834.0,95.0,109.0,107.0,888.5,148.5,0
4,b6ab77fd7,26.0,144.0,124.0,117.0,88.0,202.5,0
...,...,...,...,...,...,...,...,...
147788,5e0747034,64.0,619.0,84.0,95.0,106.0,666.5,0
147789,5e0747034,292.0,549.0,107.0,82.0,345.5,590.0,0
147790,5e0747034,134.0,228.0,141.0,71.0,204.5,263.5,0
147791,5e0747034,430.0,13.0,184.0,79.0,522.0,52.5,0


In [32]:
# index = list(set(df.image_id))
# len(index)

# source = 'train'
# if True:
#     for fold in [0]:
#         val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
#         for name, mini in tqdm.tqdm(df.groupby('image_id')):
#             if name in val_index:
#                 path2save = 'val2017/'
#             else:
#                 path2save = 'train2017/'
#             if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
#                 os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
#             with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
#                 row = mini[['classes','x_center','y_center','w','h']].astype(float).values
#                 row = row/1024
#                 row = row.astype(str)
#                 for j in range(len(row)):
#                     text = ' '.join(row[j])
#                     f.write(text)
#                     f.write("\n")
#             if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
#                 os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
#             shutil.copy("/content/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))


index = list(set(df.image_id))  #Each picture contains lots of bounding boxes, so #BBs >> #Img_ids
print(f'There are {len(index)} Images with {len(df.image_id)} Bounding Boxes')
source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        print(len(index)*fold//5, len(index)*(fold+1)//5, val_index)
        for name, mini in tqdm.tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
        #     if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
        #         os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
        #     with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
        #         row = mini[['classes','x_center','y_center','w','h']].astype(float).values
        #         row = row/1024
        #         row = row.astype(str)
        #         for j in range(len(row)):
        #             text = ' '.join(row[j])
        #             f.write(text)
        #             f.write("\n")
        #     if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
        #         os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
        #     shutil.copy("/content/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

There are 3373 Images with 147793 Bounding Boxes
0 674 ['fe3f956a2', '41457a646', '6f1e9046d', '4ecd10d42', '5b558a4c7', '461d55f4b', '16daa81b4', 'a3bc2d2ec', '9561a71a0', '854b60f75', '86e843e5e', '8201b54e5', '5a77d4b5d', '9f1d83913', 'd2e7465f0', '12f742390', 'a7e2015b3', '8a29ab483', 'e1e3b78ab', '948c989ef', '0435654cd', 'a4b64deca', '8f06b77e5', '07f519e11', '0114c88aa', '89fe13786', 'b30d051ab', '586a28040', '57956c488', '060740e25', 'ecad4b636', '70aff3f9e', 'c88635e09', 'c328e6c68', 'ff5c9104a', '26373af4b', '0e0a0483e', '4759e7258', '029e03e52', '4a06c2f96', '1a39607ca', '7c23b7b20', '4dc07d36a', 'ea3537563', '9a9f42302', 'bfaa92629', '958c0ad3d', 'e2e13ad15', 'd7a02151d', '408013a9d', '530a756df', '086349fe8', '7c79cea8f', '8afea785a', '38dc44b85', '14d856fb1', 'c29961ce8', 'f0a52d979', '2d635716b', 'cea58b404', 'f244c9c23', 'f1a038768', 'bd68a3064', '5498d5aae', '93c9d721e', '16e075fbb', 'f926234d6', 'dc75973ed', '5c64b986e', '229d8a499', 'e5ab88ff3', '6d27a58b7', 'd145202

100%|██████████| 3373/3373 [00:00<00:00, 36106.12it/s]


In [36]:
for i, j in df.groupby('image_id'):
  if i != '00333207f':
    print(j)
    break

       image_id      x      y      w      h  x_center  y_center  classes
1406  005b0d8bb  765.0  879.0  116.0   79.0     823.0     918.5        0
1407  005b0d8bb   84.0  539.0  153.0   96.0     160.5     587.0        0
1408  005b0d8bb  855.0  781.0   96.0   76.0     903.0     819.0        0
1409  005b0d8bb  798.0  934.0  133.0   88.0     864.5     978.0        0
1410  005b0d8bb  330.0  395.0  145.0  215.0     402.5     502.5        0
1411  005b0d8bb    0.0    0.0   72.0  106.0      36.0      53.0        0
1412  005b0d8bb  711.0   60.0  138.0   97.0     780.0     108.5        0
1413  005b0d8bb  157.0  393.0  220.0  108.0     267.0     447.0        0
1414  005b0d8bb  967.0  545.0   57.0  139.0     995.5     614.5        0
1415  005b0d8bb  305.0  577.0  220.0  103.0     415.0     628.5        0
1416  005b0d8bb  562.0  518.0  194.0  110.0     659.0     573.0        0
1417  005b0d8bb  737.0  555.0  123.0  181.0     798.5     645.5        0
1418  005b0d8bb  212.0  151.0  184.0  116.0     304